In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.special as sc
import xtrack as xt

# Computing real $\Delta R$

In [2]:
line = xt.Line.from_json('../injection_lines/sps_with_aperture_inj_q20_beam_sagitta5.json')

Loading line from dict:   0%|          | 0/36404 [00:00<?, ?it/s]

Done loading line from dict.           


In [3]:
tw = line.twiss()

In [24]:
# Number of bends in line = 744
tt = line.get_table()
names_arr = []
for name in tt.rows['mb.*'].name:
    names_arr.append(name[:9])
names_arr = np.array(names_arr)
names_arr = np.unique(names_arr)
print(len(names_arr))

744


In [63]:
E = tw.gamma0 * 938.27e6
c = 299792458
P = np.sqrt(E**2 - (938.27e6)**2)/c

eta = tw.slip_factor
alpha = tw.momentum_compaction_factor
gamma_tr = 1/np.sqrt(alpha)

# l_bend = 6.26
# R = 744*l_bend/(2*np.pi)
R = line.get_length()/2*np.pi

beta = np.sqrt(1 - 1/tw.gamma0**2)
V0 = 3.5e6
f0 = 200_000_000

In [64]:
DR_th = -22e-3
Df = -eta * gamma_tr**2 *f0 * DR_th/R #sign added idk why
T0 = 1.027
Tf = 1.967
a = Df/(Tf-T0)

In [65]:
S0, C0 = sc.fresnel((f0 + 2*a*T0)/np.sqrt(a))
const_0 = np.sin(np.pi * f0**2/(2*a)) * S0 - np.cos(np.pi*f0**2/(2*a)) * C0

Sf, Cf = sc.fresnel((f0 + 2*a*Tf)/np.sqrt(a))
const_f = np.sin(np.pi * f0**2/(2*a)) * Sf - np.cos(np.pi*f0**2/(2*a)) * Cf 

In [66]:
DR = P * R * gamma_tr**2 * V0 / (E * beta**2 * 2 * np.sqrt(a)) * (const_f - const_0)

In [68]:
DR

np.float64(-3.426574305266615e-05)

In [77]:
(1.967-1.027)**3/3 - (1.967-1.027)**2/2

-0.16493866666666668

In [69]:
DR_th

-0.022

In [83]:
Df_pract = -eta*gamma_tr**2*f0*DR/R *1e3

In [84]:
Df_pract

np.float64(365.06889261947015)

In [85]:
Df

np.float64(234.38906972727756)

# Mathematica series expansion of the integral

In [87]:
def get_def_int(E0, V0, f0, P, R, gamma_tr, f_rate,t_sweep):
    prefact = P*R*V0*gamma_tr**2/E0**2
    t2_fact = -np.pi * E0 * f0 * t_sweep**2
    t3_fact = -2*t_sweep**3 * np.pi*(f_rate*E0**2+2*np.pi*(E0-2)*V0*f0**2)/(3*E0**2)
    t4_fact = np.pi**2 * f0 * t_sweep**4 *(np.pi*f0**2 * (E0**3 - 6*E0*V0**2 + 12*V0**2) -6*f_rate*(E0-2)*E0*V0)/(3*E0**2)
    return prefact * (t2_fact + t3_fact + t4_fact)

In [90]:
DR = get_def_int(E, V0, P, f0, R, gamma_tr, a, 1.967) - get_def_int(E, V0, P, f0, R, gamma_tr, a, 1.027)

In [91]:
DR

np.float64(8.739098092541658e+18)

In [92]:
def eval_int_simpl(E0, V0, f0, P, R, gamma_tr, f_rate,t):
    t_term = (-2*E0+ E0**2 -2*V0*np.pi)/(np.pi*V0) * t
    t2_term = 2*f0*t**2
    t3_term = 4*f_rate/3 *t**3
    atg_term = ((-2 + E0)*E0**2 * np.atan((np.sqrt(V0*np.pi)*(f0+2*f_rate*t))/(np.sqrt(2*f_rate*E0-2*f_rate*V0*np.pi-V0*f_rate**2*np.pi))))/((V0*np.pi)**(3/2)*np.sqrt(2*f_rate*E0-2*f_rate*V0*np.pi-V0*f_rate**2*np.pi))

    return P*R*V0*gamma_tr**2*np.pi/E0**2 * (t_term + t2_term + t3_term + atg_term)

In [93]:
DR = eval_int_simpl(E, V0, f0, P, R, gamma_tr, a, 1.967) - eval_int_simpl(E, V0, f0, P, R, gamma_tr, a, 1.027)

In [94]:
DR

np.float64(284311233.6285262)

In [95]:
import xtrack as xt

In [99]:
line_lhc = xt.Line.from_json(Path('/Users/lisepauwels/sps_simulations/archive/injecion_lines/lhc_run3_b1.json'))

Loading line from dict:   0%|          | 0/101993 [00:00<?, ?it/s]

Done loading line from dict.           


In [100]:
tw = line_lhc.twiss()

In [98]:
from pathlib import Path

In [101]:
tw.gamma0

np.float64(7247.364688568272)

In [102]:
np.sqrt(1 - 1/tw.gamma0**2)

np.float64(0.9999999904805956)

In [103]:
6.8e12/938.27e6

7247.380817888241